# UBX Reader - u-blox Binary Format

Copyright 2022 Michael George (AKA Logiqx).

This file is part of [GPS Wizard](https://logiqx.github.io/gps-wizard/) and is distributed under the terms of the GNU General Public License.

GPS Wizard is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

GPS Wizard is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with Foobar. If not, see <https://www.gnu.org/licenses/>.

## Notes

This module was created because the UBXReader of pyubx2 was way too slow for my liking.

UBXReader took around 9 seconds for my test file, whereas populating numpy arrays with this module took around 0.2 seconds.

In [1]:
import os
import sys
import time

from datetime import datetime, timezone

import numpy as np

import unittest

from base_reader import BaseReader

## Main Class

In [2]:
class UbxReader(BaseReader):
    '''UBX file - u-blox Binary Format'''

    def __init__(self, filename):
        '''Basic init just records the filename'''

        super().__init__(filename)


    def load(self, ignoreChecksums=False):
        '''Load file into memory'''

        with open(self.filename, "rb") as f:
            self.buffer = f.read()
            
            # The memoryview will be used to access the buffer without intermediate copying
            self.bufferView = memoryview(self.buffer)

            if len(self.buffer) > 0:
                self.readData(ignoreChecksums=ignoreChecksums)


    def readData(self, ignoreChecksums=False):
        '''Read NAV-PVT messages into memory'''

        fileSize = len(self.buffer)

        # Lean buffer will only contain the geodetic navigation data, minus start / end tokens and checksums
        leanBuffer = bytearray()

        while self.bufferPtr < fileSize:
            msgClass, msgId, payload = self.readMessage(ignoreChecksums=ignoreChecksums)
            
            if msgClass == 0x01 and msgId == 0x07:
                # Use += because it is faster than extend - see https://stackoverflow.com/q/40004517
                leanBuffer += payload
                self.numRecords += 1
                       
        if self.bufferPtr != fileSize:
            raise ValueError('File appears to be truncated')

        if len(leanBuffer) % self.numRecords != 0:
            raise ValueError('File appears to have varying message lengths')
        messageLen = len(leanBuffer) // self.numRecords

        dtype = self.getDatatype(messageLen)
        
        self.rawData = np.frombuffer(leanBuffer, dtype=dtype, count=self.numRecords)
        
        self.consumeRawData()


    def readMessage(self, ignoreChecksums=False):
        '''Read UBX frame from buffer'''

        offset = self.bufferPtr

        syncChar1 = self.bufferView[self.bufferPtr]
        syncChar2 = self.bufferView[self.bufferPtr + 1]       
        msgClass = self.bufferView[self.bufferPtr + 2]
        msgId = self.bufferView[self.bufferPtr + 3]

        self.bufferPtr += 4

        if syncChar1 != 0xb5 or syncChar2 != 0x62:
            raise ValueError('Unrecognised sync chars - 0x{:02x} 0x{:02x}'.format(syncChar1, syncChar2))

        msgLength = self.readUnsigned16LE()       

        payload = self.readBytes(msgLength)

        expectedChecksum = self.readUnsigned16LE()

        # Evaluate checksum
        if ignoreChecksums == False:
            actualChecksum = self.calculateChecksum(self.bufferView[offset + 2 : offset + 6 + msgLength])
            if actualChecksum != expectedChecksum:
                raise ValueError('Checksum difference - {} vs {}'.format(actualChecksum, expectedChecksum))

        return msgClass, msgId, payload


    def calculateChecksum(self, message):
        '''Calculate Fletcher's checksum for the UBX message'''

        checksum_a = checksum_b = 0
        
        # Sum the message bytes
        for byte in message:
            checksum_a += byte
            checksum_b += checksum_a

        # Bit masking has been deferred until the end
        return (checksum_b & 0xff) << 8 | checksum_a & 0xff


    def getDatatype(self, messageLen):
        '''Get the datatype of the NAV-PVT payload'''

        # Standard 92-byte NAV-PVT payload
        dtype = [
            ('iTOW', '<u4'),      # GPS time of week - IGNORED
            ('year', '<u2'),      # Year (UTC)
            ('month', 'u1'),      # Month (UTC)
            ('day', 'u1'),        # Day (UTC)
            ('hour', 'u1'),       # Hour (UTC)
            ('min', 'u1'),        # Minute (UTC)
            ('sec', 'u1'),        # Second (UTC)
            ('valid', 'u1'),      # Validity flags - IGNORED
            ('tAcc', '<u4'),      # Time accuracy estimate - IGNORED
            ('nano', '<i4'),      # Fraction of second (ns)
            ('fixType', 'u1'),    # GNSS fix type (includes dead reckoning)
            ('flags', 'u1'),      # Fix status flags - IGNORED
            ('flags2', 'u1'),     # Additional flags - IGNORED
            ('numSV', 'u1'),      # Number of satellites used
            ('lon', '<i4'),       # Longitude
            ('lat', '<i4'),       # Latitude
            ('height', '<i4'),    # Height above ellipsoid - IGNORED
            ('hMSL', '<i4'),      # Height above mean sea level
            ('hAcc', '<u4'),      # Horizontal accuracy estimate (mm)
            ('vAcc', '<u4'),      # Vertical accuracy estimate (mm)
            ('velN', '<i4'),      # NED north velocity (mm/s) - IGNORED
            ('velE', '<i4'),      # NED east velocity (mm/s) - IGNORED
            ('velD', '<i4'),      # NED down velocity (mm/s)
            ('gSpeed', '<i4'),    # Ground Speed (2-D, mm/s)
            ('headMot', '<i4'),   # Heading of motion (2-D, degrees)
            ('sAcc', '<u4'),      # Speed accuracy estimate (mm/s)
            ('headAcc', '<u4'),   # Heading accuracy estimate (both motion and vehicle, degrees)
            ('pDOP', '<u2'),      # Position DOP
            ('flags3', '<u2'),    # Additional flags - IGNORED
            ('reserved', '<u4'),  # Reserved - IGNORED
            ('headVeh', '<i4'),   # Heading of vehicle - IGNORED
            ('magDec', '<i2'),    # Magnetic declination - IGNORED
            ('magAcc', '<u2')     # Magnetic declination accuracy - IGNORED
        ]

        # Forward compatibility / future-proofing in case NAV-PVT messages exceed 92 bytes in length
        for i in range(messageLen - 92):
            dtype.extend([
                ('unknown{:2}'.format(i), 'u1')
            ])

        return np.dtype(dtype)


    def consumeRawData(self):
        '''Consume data - convert to standard data structure'''

        # Convert latitude and longitude to decimal
        self.data['lat'] = np.divide(self.rawData['lat'], 10000000, dtype='float64')
        self.data['lon'] = np.divide(self.rawData['lon'], 10000000, dtype='float64')

        # Convert elevation from cm to m
        self.data['ele'] = np.divide(self.rawData['hMSL'], 1000, dtype='float64')

        # Convert SOG from mm/s to m/s
        self.data['sog'] = np.divide(self.rawData['gSpeed'], 1000, dtype='float32')

        # Convert COG from integer to decimal
        self.data['cog'] = np.round(np.divide(self.rawData['headMot'], 100000, dtype='float32'), 3)

        # Convert ROC from mm/s to m/s
        self.data['roc'] = np.divide(self.rawData['velD'], -1000, dtype='float32')

        # Convert date/time to regular timestamp (seconds)
        vfunc = np.vectorize(getDateTime)
        self.data['ts'] = vfunc( self.rawData['year'], self.rawData['month'], self.rawData['day'],
                                        self.rawData['hour'], self.rawData['min'], self.rawData['sec'], self.rawData['nano'])

        # Copy fix and satellites
        self.data['fix'] = self.rawData['fixType']
        self.data['sat'] = self.rawData['numSV']

        # Convert sAcc from mm/s to m/s
        self.data['sacc'] = np.divide(self.rawData['sAcc'], 1000, dtype='float32')

        # Convert hAcc and vAcc from mm to m
        self.data['hacc'] = np.divide(self.rawData['hAcc'], 1000, dtype='float32')
        self.data['vacc'] = np.divide(self.rawData['vAcc'], 1000, dtype='float32')
        
        # Convert cAcc to degrees
        self.data['cacc'] = np.round(np.divide(self.rawData['headAcc'], 100000, dtype='float32'), 3)

        # Convert PDOP from integer to decimal
        self.data['pdop'] = np.divide(self.rawData['pDOP'], 100, dtype='float32')
            

def getDateTime(year, month, day, hour, minute, second, nano):
    '''Decode date/time'''

    timestamp = datetime(year, month, day, hour, minute, second, tzinfo=timezone.utc).timestamp()

    # Note that nano can be positive or negative, hence the adhoc adjustment
    return timestamp + np.round(nano / 1000000000, 3)

## Unit Tests

In [3]:
class TestEspData(unittest.TestCase):
    '''Class to test ESP-GPS data was correctly loaded'''

    def testNumRecords(self):
        '''Test the number of records is as expected'''

        self.assertEqual(espReader.numRecords, 43444)

        for fieldName in espReader.data:
            self.assertEqual(espReader.data[fieldName].size, 43444)


    def testPdop(self):
        '''Test the positional dilution of precision is as expected'''

        self.assertEqual(espReader.data['pdop'].min(), np.float32(0.96))
        self.assertEqual(espReader.data['pdop'].max(), np.float32(1.88))


    def testSat(self):
        '''Test the satellite count is as expected'''

        self.assertEqual(espReader.data['sat'].min(), np.uint8(9))
        self.assertEqual(espReader.data['sat'].max(), np.uint8(18))


    def testTimestamp(self):
        '''Test the timestamp is as expected'''

        self.assertEqual(espReader.data['ts'].min(), np.float64(1640864825.3))
        self.assertEqual(espReader.data['ts'].max(), np.float64(1640869169.6))


    def testLat(self):
        '''Test the latitude is as expected'''

        self.assertEqual(espReader.data['lat'].min(), np.float64(51.6993591))
        self.assertEqual(espReader.data['lat'].max(), np.float64(51.7087559))


    def testLon(self):
        '''Test the longitude is as expected'''

        self.assertEqual(espReader.data['lon'].min(), np.float64(4.0860906))
        self.assertEqual(espReader.data['lon'].max(), np.float64(4.0890658))


    def testEle(self):
        '''Test the elevation is as expected'''

        self.assertEqual(espReader.data['ele'].min(), np.float64(-3.159))
        self.assertEqual(espReader.data['ele'].max(), np.float64(11.978))


    def testSog(self):
        '''Test the speed over ground is as expected'''

        self.assertEqual(espReader.data['sog'].min(), np.float32(0.000))
        self.assertEqual(espReader.data['sog'].max(), np.float32(18.169))


    def testRoc(self):
        '''Test the rate of climb is as expected'''

        self.assertEqual(espReader.data['roc'].min(), np.float32(-1.715))
        self.assertEqual(espReader.data['roc'].max(), np.float32(1.036))


    def testCog(self):
        '''Test the course over ground is as expected'''

        self.assertEqual(espReader.data['cog'].min(), np.float32(0.011))
        self.assertEqual(espReader.data['cog'].max(), np.float32(359.995))


    def testFix(self):
        '''Test the fix is as expected'''

        self.assertEqual(espReader.data['fix'].min(), np.uint8(3))
        self.assertEqual(espReader.data['fix'].max(), np.uint8(3))


    def testSacc(self):
        '''Test the speed accuracy is as expected'''

        self.assertEqual(espReader.data['sacc'].min(), np.float32(0.051))
        self.assertEqual(espReader.data['sacc'].max(), np.float32(0.771))


    def testHacc(self):
        '''Test the horizontal accuracy is as expected'''

        self.assertEqual(espReader.data['hacc'].min(), np.float32(0.169))
        self.assertEqual(espReader.data['hacc'].max(), np.float32(2.508))


    def testVacc(self):
        '''Test the vertical accuracy is as expected'''

        self.assertEqual(espReader.data['vacc'].min(), np.float32(0.207))
        self.assertEqual(espReader.data['vacc'].max(), np.float32(2.582))


    def testCacc(self):
        '''Test the course accuracy is as expected'''

        self.assertEqual(espReader.data['cacc'].min(), np.float32(0.239))
        self.assertEqual(espReader.data['cacc'].max(), np.float32(44.999))

In [4]:
if __name__ == '__main__':
    projdir = os.path.realpath(os.path.join(sys.path[0], "..", ".."))

    filename = os.path.join(projdir, 'sessions', '20211230-esp', 'Head_L_7C9EBDFAF5C8_007.ubx')
    espReader = UbxReader(filename)

    pc1 = time.perf_counter()
    espReader.load()
    pc2 = time.perf_counter()

    print("\nUBX file loaded in in %0.2f seconds" % (pc2 - pc1))


UBX file loaded in in 0.62 seconds


In [5]:
if __name__ == '__main__':
    # Determine whether session is interactive or batch to facilitate unittest.main(..., exit=testExit)
    import __main__ as main
    testExit = hasattr(main, '__file__')

    unittest.main(argv=['first-arg-is-ignored'], exit=testExit)

...............
----------------------------------------------------------------------
Ran 15 tests in 0.010s

OK
